In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
%matplotlib widget
import pprint
import seaborn as sns 
sns.set()
#sns.reset_orig


## Part 1: Getting the Data
##




In [2]:
#######################
import myauz
from myauz.myalpha_funcs import (
    read_data,
    persist_data,
    update_csv,
    compose_portfolio,
    retrieveDF,
    string2date,
    retrievePF,
    initialize_df,
    time_sleep,
    create_path_list,
    refresh_db,
    get_daily_symbol,
)

from myauz.myalpha_optimize import (
    pf_filtered,
    symbol_list_filtered,
    determine_earliest_notation,
    calc_weights,
    get_ret_vol_sr,
    neg_sharpe,
    check_sum,
    minimize_volatility,
)


#import importlib
#importlib.reload(myauz.myalpha_funcs)

from myauz.myalpha_libs_universal import StocksDb

import pprint


from datetime import date
import datetime
import pandas as pd



In [3]:
alphaDB = StocksDb()
alphaDB.check_path()
print("\n", alphaDB.api_key_alpha)


api_key_alpha = alphaDB.api_key_alpha
root_path = alphaDB.path
print('root_path: ',root_path)

read_from_alphavantage = False
rename_column = True


usecols = ["timestamp", "adjusted_close"]
startd = "2015-01-01"
endd = "2020-10-09"

print('usecols:', usecols)
print('startd',startd)
print('endd',endd)


Directory  ../data  already exists

 77K8XPSR3XWWZJ80
root_path:  ..
usecols: ['timestamp', 'adjusted_close']
startd 2015-01-01
endd 2020-10-09


In [4]:
symbol_list = [
   "MSFT",
#  "GOOG",    
   "DHR",
#  "EMR",
   "ATR",
   "HXGBY",
   "NVS",
   "NVZMY",    
#  "CL",
   "MCD",    
#  "HEINY",
#  "LDSVF",  
]

path_list = create_path_list(symbol_list, root_path)
pprint.pprint(path_list)

bool_list_all = [
   True, #"MSFT", microsoft    
#  True, #"GOOG", alphabet google      
#  True, #"AMZN", amazon
   True, #"DHR", danaher
#  True, #"EMR", emerson    
   True, #"ATR", aptargroup
   True, #"HXGBY", hexagon
   True, #"NVS", novartis
   True, #"NVZMY", novozymes    
#  True, #"CL", colgate palmolive
   True, #"MCD", mcdonalds    
#  True, #"HEINY", heineken
#  True, #"LDSVF", lindt & spruengli    
]

bool_list_high_tech = [
   True, #"MSFT", microsoft    
#  True, #"GOOG", alphabet google      
#  True, #"AMZN", amazon
   False, #"DHR", danaher
#  False, #"EMR", emerson    
   False, #"ATR", aptargroup
   False, #"HXGBY", hexagon
   False, #"NVS", novartis
   False, #"NVZMY", novozymes    
#  False, #"CL", colgate palmolive
   False, #"MCD", mcdonalds    
#  False, #"HEINY", heineken
#  False, #"LDSVF", lindt & spruengli    
]

bool_list_industry = [
   False, #"MSFT", microsoft    
#  False, #"GOOG", alphabet google      
#  False, #"AMZN", amazon
   True, #"DHR", danaher
#  True, #"EMR", emerson    
   True, #"ATR", aptargroup
   True, #"HXGBY", hexagon
   True, #"NVS", novartis
   True, #"NVZMY", novozymes    
#  False, #"CL", colgate palmolive
   False, #"MCD", mcdonalds    
#  False, #"HEINY", heineken
#  False, #"LDSVF", lindt & spruengli    
]

bool_list_consumer = [
   False, #"MSFT", microsoft    
#  False, #"GOOG", alphabet google      
#  False, #"AMZN", amazon
   False, #"DHR", danaher
#  False, #"EMR", emerson    
   False, #"ATR", aptargroup
   False, #"HXGBY", hexagon
   False, #"NVS", novartis
   False, #"NVZMY", novozymes    
#  True, #"CL", colgate palmolive
   True, #"MCD", mcdonalds    
#  True, #"HEINY", heineken
#  True, #"LDSVF", lindt & spruengli    
]

colors = [
    'magenta',
#   'lime',
#   'tab:orange',
    'blue',
#   'black',
    'slategrey',
    'tab:brown',
    'tab:olive',
    'darkviolet',
#   'red',
    'turquoise',
#   'forestgreen',
#   'gold',
]

color_dict = dict(zip(symbol_list, colors))
print('\n')
print(color_dict)


{'ATR': '../data/ATR/daily_ATR.csv',
 'DHR': '../data/DHR/daily_DHR.csv',
 'HXGBY': '../data/HXGBY/daily_HXGBY.csv',
 'MCD': '../data/MCD/daily_MCD.csv',
 'MSFT': '../data/MSFT/daily_MSFT.csv',
 'NVS': '../data/NVS/daily_NVS.csv',
 'NVZMY': '../data/NVZMY/daily_NVZMY.csv'}


{'MSFT': 'magenta', 'DHR': 'blue', 'ATR': 'slategrey', 'HXGBY': 'tab:brown', 'NVS': 'tab:olive', 'NVZMY': 'darkviolet', 'MCD': 'turquoise'}


In [5]:
# filter columns(symbols) in dataframe  by bool_list(symbols = True - this is the _included list)
def pf_filtered(_pf, _startd, _included):
    if not (_startd):
        _startd = '2020-01-01'
    if _included is []:
        return _pf
    else: 
        return _pf.loc[startd:,_included]

from itertools import compress
def symbol_list_filtered(_symbol_list,_included):
    #filter the symbol_list
    #symbol_list_filtered = [i for indx,i in enumerate(symbol_list) if included[indx]]
    return list(compress(_symbol_list, _included))

def determine_earliest_notation(_pf):
    """
    input:
    ------
         portfolio notations - portfolio-columns are also symbols
    
    output:
    -------
         dataframe with one row containing earliest notation in time of symbol in portfolio
    
    """
    _pf_tmp = _pf.copy()
    _pf_tmp.fillna(0, inplace=True)
    _symbol_list = list(_pf_tmp.columns)
    _df_aux_0 = _pf_tmp.iloc[0]
    
    for symbol in _symbol_list:
        _df_aux_0[symbol] = (_pf_tmp[_pf_tmp[symbol] > 0].iloc[0]).loc[symbol]  
    return _df_aux_0

In [15]:
print('root_path:',root_path)

root_path: ..


In [16]:
print('root_path:',root_path)
### create dataframe with portfolio adjusted close
usecols = ["timestamp", "adjusted_close"]
path_list = create_path_list(symbol_list, root_path)
pprint.pprint(path_list)
pf = retrievePF(symbol_list, path_list, startd, endd, usecols, rename_column)
print(pf.index.name)
pf.index.rename('DATE', inplace=True)
print(pf.index.name)
pf.fillna(method='ffill', inplace=True)
pf

root_path: ..
{'ATR': '../data/ATR/daily_ATR.csv',
 'DHR': '../data/DHR/daily_DHR.csv',
 'HXGBY': '../data/HXGBY/daily_HXGBY.csv',
 'MCD': '../data/MCD/daily_MCD.csv',
 'MSFT': '../data/MSFT/daily_MSFT.csv',
 'NVS': '../data/NVS/daily_NVS.csv',
 'NVZMY': '../data/NVZMY/daily_NVZMY.csv'}
None
DATE


,MSFT,DHR,ATR,HXGBY,NVS,NVZMY,MCD
DATE,,,,,,,
2015-01-02,41.6513,63.3710,61.2037,28.5702,75.3810,39.0758,79.6897
2015-01-05,41.2638,62.7426,59.3212,28.1195,75.5445,38.0205,78.8096
2015-01-06,40.6625,62.3139,58.6876,28.0091,74.9071,38.4268,78.9548
2015-01-07,41.1792,62.6909,59.0274,27.5032,75.2339,37.8897,80.3306
2015-01-08,42.3906,63.6594,60.7078,27.5032,77.8488,38.9171,80.6296
...,...,...,...,...,...,...,...
2020-10-02,206.1900,212.5000,112.8500,75.1500,87.5200,62.7500,222.6700
2020-10-05,210.3800,218.3500,114.5500,76.5400,88.2700,63.7600,226.0700
2020-10-06,205.9100,218.2400,114.5600,74.9250,86.4500,62.1900,224.0900


In [17]:
df_returns = (pf/pf.shift(1))-1
#ret = pf.pct_change(1)
pprint.pprint(df_returns.head())

print('___________________')

df1_returns = pf_filtered(df_returns, startd, bool_list_high_tech)


print('\n high_tech:')
#df1_returns = (pf1/pf1.shift(1))-1
pprint.pprint(df1_returns.head(3))


print('\n industry:')
#df2_returns = (pf2/pf2.shift(1))-1
df2_returns = pf_filtered(df_returns, startd, bool_list_industry)
pprint.pprint(df2_returns.head(3))


print('\n consumer: ')
#df3_returns = (pf3/pf3.shift(1))-1
df3_returns = pf_filtered(df_returns, startd, bool_list_consumer)
pprint.pprint(df3_returns.head(3))

                MSFT       DHR       ATR     HXGBY       NVS     NVZMY  \
DATE                                                                     
2015-01-02       NaN       NaN       NaN       NaN       NaN       NaN   
2015-01-05 -0.009303 -0.009916 -0.030758 -0.015775  0.002169 -0.027006   
2015-01-06 -0.014572 -0.006833 -0.010681 -0.003926 -0.008437  0.010686   
2015-01-07  0.012707  0.006050  0.005790 -0.018062  0.004363 -0.013977   
2015-01-08  0.029418  0.015449  0.028468  0.000000  0.034757  0.027116   

                 MCD  
DATE                  
2015-01-02       NaN  
2015-01-05 -0.011044  
2015-01-06  0.001842  
2015-01-07  0.017425  
2015-01-08  0.003722  
___________________

 high_tech:
                MSFT
DATE                
2015-01-02       NaN
2015-01-05 -0.009303
2015-01-06 -0.014572

 industry:
                 DHR       ATR     HXGBY       NVS     NVZMY
DATE                                                        
2015-01-02       NaN       NaN       NaN       N

In [18]:
df_cum_returns = initialize_df(symbol_list, startd, endd)
#print(df_cum_returns.head())
#print(df_returns)

df_cum_returns = (1+df_returns).cumprod()
###############achtung dataframe wird auf naechtes datum ohne NaN gesetzt
#df_cum_returns.dropna(inplace=True)
#################
df_cum_returns.head()
#df_cum_returns.head()

,MSFT,DHR,ATR,HXGBY,NVS,NVZMY,MCD
DATE,,,,,,,
2015-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-05,0.990697,0.990084,0.969242,0.984225,1.002169,0.972994,0.988956
2015-01-06,0.976260,0.983319,0.958890,0.980361,0.993713,0.983391,0.990778
2015-01-07,0.988665,0.989268,0.964442,0.962653,0.998049,0.969646,1.008042
2015-01-08,1.017750,1.004551,0.991898,0.962653,1.032738,0.995939,1.011794


In [19]:
invested_dict = {
    "MSFT": float(1297.59),
    "GOOG": float(2111),
    "DHR": float(3168.48),
    "EMR": float(418.10),
    "ATR": float(289.49),
    "HXGBY": float(1060.39),
    "NVS": float(404.42),
    "NVZMY": float(1049.95),
    "CL": float(390.03),
    "MCD": float(872.35),  
    "HEINY": float(575.60),     
    "LDSVF": float(6942.25),   
}

invested_dict = {
    "MSFT": float(1297.59),
#   "GOOG": float(2111),
    "DHR": float(3168.48),
#   "EMR": float(418.10),
    "ATR": float(289.49),
    "HXGBY": float(1060.39),
    "NVS": float(404.42),
    "NVZMY": float(1049.95),
#   "CL": float(390.03),
    "MCD": float(872.35),  
#   "HEINY": float(575.60),     
#   "LDSVF": float(6942.25),   
}

In [20]:
total_investment, weights_dict = calc_weights(invested_dict)

print("total investment:",total_investment)

weight_sum = 0
for symbol, weight in weights_dict.items():
    pprint.pprint((symbol, weight))
    weight_sum += weight

print(weight_sum)

"""
weights = [
   .1,  #"MSFT",
   .1,  #"GOOG",    
   .1, #"DHR",
   .1,  #"EMR",
   .1,  #"ATR",
   .1,  #"HXGBY",
   .1,  #"NVS",
   .1,  #"NVZMY",    
   .1,  #"CL",
   .05,  #"MCD",    
   .03,  #"HEINY",
   .02,  #"LDSVF",  
]
"""

total investment: 8142.67
('MSFT', 0.15935682030587017)
('DHR', 0.38912052189269614)
('ATR', 0.03555222058612224)
('HXGBY', 0.1302263262541648)
('NVS', 0.049666755499117615)
('NVZMY', 0.1289441915244017)
('MCD', 0.10713316393762734)
1.0


'\nweights = [\n   .1,  #"MSFT",\n   .1,  #"GOOG",    \n   .1, #"DHR",\n   .1,  #"EMR",\n   .1,  #"ATR",\n   .1,  #"HXGBY",\n   .1,  #"NVS",\n   .1,  #"NVZMY",    \n   .1,  #"CL",\n   .05,  #"MCD",    \n   .03,  #"HEINY",\n   .02,  #"LDSVF",  \n]\n'

In [21]:
#ensure that weights vector is in correct order as symbol_list
wlist = []
for symbol in symbol_list:
    print(symbol)
    wlist.append(weights_dict[symbol])

print('wlist',wlist)
#weights = np.array(list(weights_dict.values()))
actual_weights=np.array(wlist)

print('\n',actual_weights.sum())


pf_cum_ret = df_cum_returns * actual_weights * total_investment
pf_cum_ret.fillna(method='ffill', inplace=True, axis=1)
pf_cum_ret['TOTAL_POS'] = pf_cum_ret.sum(axis=1)

print('\n _________________________________________________________________________________')

pf_cum_ret = pf_cum_ret[1:]

pf_cum_ret['TOTAL_POS']
pf_cum_ret.to_csv('../data/pf_cum_ret.csv')

MSFT
DHR
ATR
HXGBY
NVS
NVZMY
MCD
wlist [0.15935682030587017, 0.38912052189269614, 0.03555222058612224, 0.1302263262541648, 0.049666755499117615, 0.1289441915244017, 0.10713316393762734]

 1.0

 _________________________________________________________________________________


In [29]:
plt.figure()
plt.title("total position of portfolio")
# Turn on the minor TICKS, which are required for the minor GRID
plt.minorticks_on()
# Customize the major grid
plt.grid(which='major', linestyle='-', linewidth='0.5', color='dimgray')
# Customize the minor grid
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='gray')





pf_cum_ret['TOTAL_POS'].plot(color=['royalblue'], figsize=(16,10), linewidth=1.25);


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [30]:
pf_cum_ret.head()
color_list = [color_dict.get(x, _) for x in pf_cum_ret.columns]
print(pf_cum_ret.iloc[0])
#pf_cum_ret.fillna(method='ffill', inplace=True)
_pf_cum_ret  = pf_cum_ret / pf_cum_ret.iloc[0]

_pf_cum_ret[symbol_list].plot(color=color_list,figsize=(16,10), linewidth=1.25);


plt.title('all stocks in portfolio')
# Turn on the minor TICKS, which are required for the minor GRID
plt.minorticks_on()
# Customize the major grid
plt.grid(which='major', linestyle='-', linewidth='0.5', color='dimgray')
# Customize the minor grid
plt.grid( which='minor', linestyle=':', linewidth='0.5', color='gray')





MSFT         1285.517961
DHR          3137.060694
ATR           280.585883
HXGBY        1043.662159
NVS           405.297180
NVZMY        1021.594541
MCD           862.715690
TOTAL_POS    8036.434108
Name: 2015-01-05 00:00:00, dtype: float64


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [52]:
plt.close('all')

color_list_nvs = [color_dict.get('NVS', _)]
plt.title('all stocks in portfolio')
# Turn on the minor TICKS, which are required for the minor GRID
plt.minorticks_on()
# Customize the major grid
plt.grid(which='major', linestyle='-', linewidth='0.5', color='dimgray')
# Customize the minor grid
plt.grid( which='minor', linestyle=':', linewidth='0.5', color='gray')
_pf_cum_ret['NVS'].plot(color=color_list_nvs,figsize=(10,8), linewidth=1.25);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Sharpe Ratio

In [33]:
pf_cum_ret['DAILY_RETURN']= pf_cum_ret['TOTAL_POS'].pct_change(1)

print('\nvolatility daily:\t',pf_cum_ret['DAILY_RETURN'].std())
print('return daily: \t\t', pf_cum_ret['DAILY_RETURN'].mean())

actual_annual_volatility = pf_cum_ret['DAILY_RETURN'].std() * 250 **0.5
actual_annual_return = pf_cum_ret['DAILY_RETURN'].mean() * 250

print('\n\nvolatility annually:\t',actual_annual_volatility )
print('return annually:\t', actual_annual_return )
      
cum_return = 100 * ( ( pf_cum_ret['TOTAL_POS'][-1] - pf_cum_ret['TOTAL_POS'][0] ) / pf_cum_ret['TOTAL_POS'][0] )
print('\naccumulated return: \t',cum_return)


#calculate sharpe ratio daily assume risk-free rate to be zero
actual_sharp_ratio = pf_cum_ret['DAILY_RETURN'].mean() / pf_cum_ret['DAILY_RETURN'].std()

# annualize Sharpe Ratio 
# annual sharpe ratio (asr) should be above 1 


print('\nannual sharp ratio should be above 1')
actual_sharp_ratio_annual = (252**0.5)*actual_sharp_ratio

print('\nsharp_ratio:\t\t', actual_sharp_ratio)

print('annual_sharp_ratio:\t',actual_sharp_ratio_annual)

              




volatility daily:	 0.011671088805965499
return daily: 		 0.0008660828675565877


volatility annually:	 0.18453611700472972
return annually:	 0.21652071688914692

accumulated return: 	 218.05224678451668

annual sharp ratio should be above 1

sharp_ratio:		 0.07420754669554931
annual_sharp_ratio:	 1.1780082837638186


In [34]:
plt.close('all')
plt.figure()
plt.title("total position of portfolio")
# Turn on the minor TICKS, which are required for the minor GRID
plt.minorticks_on()
# Customize the major grid
plt.grid(which='major', linestyle='-', linewidth='0.5', color='dimgray')
# Customize the minor grid
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='gray')

pf_cum_ret['DAILY_RETURN'].plot(kind='kde', figsize=(10,8), xlim=[-.05,.05]);
pf_cum_ret


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

,MSFT,DHR,ATR,HXGBY,NVS,NVZMY,MCD,TOTAL_POS,DAILY_RETURN
DATE,,,,,,,,,
2015-01-05,1285.517961,3137.060694,280.585883,1043.662159,405.297180,1021.594541,862.715690,8036.434108,NaN
2015-01-06,1266.785271,3115.626168,277.588991,1039.564635,401.877521,1032.511648,864.305171,7998.259405,-0.004750
2015-01-07,1282.882362,3134.475751,279.196225,1020.788033,403.630807,1018.080001,879.365827,8018.419006,0.002520
2015-01-08,1320.621893,3182.899681,287.144421,1020.788033,417.659778,1045.685799,882.638930,8157.438535,0.017338
2015-01-09,1309.521848,3161.460155,284.364634,1044.686539,421.386315,1041.169024,871.882570,8134.471086,-0.002816
...,...,...,...,...,...,...,...,...,...
2020-10-02,6423.570983,10624.765271,533.774045,2789.210734,469.545886,1686.065608,2437.531758,24964.464284,-0.012811
2020-10-05,6554.104775,10917.258809,541.814947,2840.800925,473.569645,1713.203876,2474.750997,25515.503974,0.022073
2020-10-06,6414.847962,10911.758931,541.862247,2780.859803,463.805322,1671.018648,2453.076263,25237.229176,-0.010906


# portfolio optimization
# - monte carlo
# - mathematical optimization by minimizing the negative sharpe ratio

In [35]:
pf


,MSFT,DHR,ATR,HXGBY,NVS,NVZMY,MCD
DATE,,,,,,,
2015-01-02,41.6513,63.3710,61.2037,28.5702,75.3810,39.0758,79.6897
2015-01-05,41.2638,62.7426,59.3212,28.1195,75.5445,38.0205,78.8096
2015-01-06,40.6625,62.3139,58.6876,28.0091,74.9071,38.4268,78.9548
2015-01-07,41.1792,62.6909,59.0274,27.5032,75.2339,37.8897,80.3306
2015-01-08,42.3906,63.6594,60.7078,27.5032,77.8488,38.9171,80.6296
...,...,...,...,...,...,...,...
2020-10-02,206.1900,212.5000,112.8500,75.1500,87.5200,62.7500,222.6700
2020-10-05,210.3800,218.3500,114.5500,76.5400,88.2700,63.7600,226.0700
2020-10-06,205.9100,218.2400,114.5600,74.9250,86.4500,62.1900,224.0900


In [36]:
# annual mean of returns as %
type(pf.pct_change(1).mean()*252)
(pf.pct_change(1).mean()*252).sort_values(ascending=False)


MSFT     0.320292
DHR      0.238905
HXGBY    0.225316
MCD      0.206406
ATR      0.133535
NVZMY    0.116119
NVS      0.047054
dtype: float64

In [53]:
# we use log returns for algorithms especially for normalizations
log_ret= np.log(pf/pf.shift(1))

log_ret.head()

log_ret.fillna(value=0.0,inplace=True)
log_ret.to_csv('../data/log_ret2.csv')

In [54]:
(log_ret.mean()*252).sort_values(ascending=False)

MSFT     0.281056
DHR      0.216419
MCD      0.180633
HXGBY    0.169352
ATR      0.111667
NVZMY    0.079081
NVS      0.025935
dtype: float64

In [55]:
# annual variance
print((log_ret.var()*252).sort_values(ascending=False))

# annual standard deviation
print((log_ret.std()*252**0.5).sort_values(ascending=False))

HXGBY    0.112175
MSFT     0.077755
NVZMY    0.074429
MCD      0.051138
DHR      0.044443
ATR      0.043524
NVS      0.042389
dtype: float64
HXGBY    0.334925
MSFT     0.278847
NVZMY    0.272817
MCD      0.226136
DHR      0.210814
ATR      0.208624
NVS      0.205887
dtype: float64


In [56]:
log_ret.corr() 

,MSFT,DHR,ATR,HXGBY,NVS,NVZMY,MCD
MSFT,1.000000,0.612498,0.479401,0.416623,0.493422,0.288038,0.516133
DHR,0.612498,1.000000,0.503000,0.367944,0.525469,0.256442,0.509301
ATR,0.479401,0.503000,1.000000,0.354706,0.420477,0.263835,0.461455
HXGBY,0.416623,0.367944,0.354706,1.000000,0.330939,0.258013,0.352299
NVS,0.493422,0.525469,0.420477,0.330939,1.000000,0.320095,0.405443
NVZMY,0.288038,0.256442,0.263835,0.258013,0.320095,1.000000,0.182502
MCD,0.516133,0.509301,0.461455,0.352299,0.405443,0.182502,1.000000


In [57]:
log_ret.hist(bins=20, figsize=(10,10));

#type(hist)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [58]:
np.random.seed(101)
# to keep getting the same random weights

print(pf.columns)
num_weights = len(log_ret.columns)
weights = np.array(np.random.random(num_weights))
pprint.pprint(weights)
print('sum:',np.sum(weights))
# but they do not add up to 1

print('\n rebalance: \n')
weights=weights/np.sum(weights)
print(weights)
np.sum(weights)

Index(['MSFT', 'DHR', 'ATR', 'HXGBY', 'NVS', 'NVZMY', 'MCD'], dtype='object')
array([0.51639863, 0.57066759, 0.02847423, 0.17152166, 0.68527698,
       0.83389686, 0.30696622])
sum: 3.1132021612794123

 rebalance: 

[0.16587379 0.18330566 0.00914628 0.05509493 0.22011965 0.26785824
 0.09860144]


1.0000000000000002

In [59]:
#
print('expected portfolio return')
some_exp_ret = np.sum((log_ret.mean() * weights) * 252)
print((log_ret.mean() * weights) * 252)
some_exp_ret
print(some_exp_ret)
print('\n expected volatility')
#denominator of sharp-ratio

some_exp_vol = np.sqrt(np.dot((weights.T),np.dot(log_ret.cov()*252, weights)))
print(some_exp_vol)

some_sr = some_exp_ret / some_exp_vol
print('\n sharpe ratio: ')
print(some_sr)

expected portfolio return
MSFT     0.046620
DHR      0.039671
ATR      0.001021
HXGBY    0.009330
NVS      0.005709
NVZMY    0.021182
MCD      0.017811
dtype: float64
0.1413444181697466

 expected volatility
0.17315696471753453

 sharpe ratio: 
0.816279139567486



## determine number of iterations in monte carlo

In [60]:
import numpy as np

np.random.seed(101)

num_pf = 10000

num_weights = len(log_ret.columns)
all_weights = np.zeros((num_pf, num_weights))
ret_arr = np.zeros(num_pf)
vol_arr = np.zeros(num_pf)
sharpe_arr = np.zeros(num_pf)

for ind in range(num_pf):
    _weights = np.array(np.random.random(num_weights))
    _weights = _weights / np.sum(_weights)
    all_weights[ind,:] = _weights
    # expected return
    ret_arr[ind] = np.sum((log_ret.mean() * _weights) * 252)
    #expected volatility
    vol_arr[ind] = np.sqrt(np.dot((_weights.T),np.dot(log_ret.cov()*252, _weights)))
    #sharpe ratio
    sharpe_arr[ind] = ret_arr[ind] / vol_arr[ind]
    
print('done')

done


In [61]:
print(sharpe_arr.max())
mc_sharp_ratio_annual = sharpe_arr.max()
sharpe_arr


1.111774624964584


array([0.81627914, 1.01029558, 0.89838543, ..., 0.90962634, 0.90544267,
       0.86465732])

In [62]:
sharp_argmax = sharpe_arr.argmax()
sharp_argmax

8069

In [63]:
plt.figure(figsize=(12,8))
plt.scatter(vol_arr,ret_arr, c= sharpe_arr, cmap='plasma')
plt.colorbar(label='Sharpe Ratio')
plt.xlabel('Volatility')
plt.ylabel('return')



# Turn on the minor TICKS, which are required for the minor GRID
plt.minorticks_on()
# Customize the major grid
plt.grid(which='major', linestyle='-', linewidth='0.5', color='dimgray')
# Customize the minor grid
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='gray')


max_sr_ret = ret_arr[sharp_argmax]
max_sr_vol = vol_arr[sharp_argmax]
plt.scatter(max_sr_vol, max_sr_ret,  c='blue')
plt.scatter(actual_annual_volatility, actual_annual_return,  c='red')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [64]:
log_ret.head()

,MSFT,DHR,ATR,HXGBY,NVS,NVZMY,MCD
DATE,,,,,,,
2015-01-02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2015-01-05,-0.009347,-0.009966,-0.031241,-0.015901,0.002167,-0.027378,-0.011106
2015-01-06,-0.014679,-0.006856,-0.010738,-0.003934,-0.008473,0.010630,0.001841
2015-01-07,0.012627,0.006032,0.005773,-0.018227,0.004353,-0.014076,0.017275
2015-01-08,0.028993,0.015331,0.028070,0.000000,0.034167,0.026754,0.003715


# Machine Learning Approach 

In [65]:
from scipy.optimize import minimize 
import numpy as np


In [66]:
#helper functions

def get_ret_vol_sr(weights):
    weights = np.array(weights)
    ret = np.sum(log_ret.mean() * weights)  * 252
    vol = np.sqrt(np.dot(weights.T, np.dot(log_ret.cov()*252,weights)))
    sr = ret / vol
    return np.array([ret,vol,sr])

#minimize negative sharpe ratio which is the same as maximize sr
def neg_sharpe(weights):
    return get_ret_vol_sr(weights)[2] * -1

def check_sum(weights):
    #return 0 if the sum of the weights is 1
    return np.sum(weights) - 1

def minimize_volatility(weights):
    return get_ret_vol_sr(weights)[1]

In [86]:
"""
#constraint in form of dictionary
cons = ({'type': 'eq', 'fun':check_sum})
#defines a constraint for minimize function - it says
#{Constraint, dict} or List of {Constraint, dict}
# type will be equations and a function

bounds = ((0,1),(0,1), (0,1),(0,1),(0,1),(0,1), (0,1))
#bounds : sequence or `Bounds`, optional
#        Bounds on variables for L-BFGS-B, TNC, SLSQP and
#        trust-constr methods. There are two ways to specify the bounds:
#    
#           1. Instance of `Bounds` class.
#           2. Sequence of ``(min, max)`` pairs for each element in `x`. None
#               is used to specify no bound.



#sum of weights is 1 
init_guess = [0.25,0.25,0.25,0.25, 0.25,0.25,0.25]
"""

"\n#constraint in form of dictionary\ncons = ({'type': 'eq', 'fun':check_sum})\n#defines a constraint for minimize function - it says\n#{Constraint, dict} or List of {Constraint, dict}\n# type will be equations and a function\n\nbounds = ((0,1),(0,1), (0,1),(0,1),(0,1),(0,1), (0,1))\n#bounds : sequence or `Bounds`, optional\n#        Bounds on variables for L-BFGS-B, TNC, SLSQP and\n#        trust-constr methods. There are two ways to specify the bounds:\n#    \n#           1. Instance of `Bounds` class.\n#           2. Sequence of ``(min, max)`` pairs for each element in `x`. None\n#               is used to specify no bound.\n\n\n\n#sum of weights is 1 \ninit_guess = [0.25,0.25,0.25,0.25, 0.25,0.25,0.25]\n"

In [87]:
cons = ({'type': 'eq', 'fun':check_sum})
_num_symbols = len(pf.columns)
bounds = [(0,1) for x in pf.columns] 
init_guess = [0.25 for x in pf.columns]
print(bounds)
print(init_guess)

[(0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1)]
[0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25]


In [88]:
opt_results = minimize(neg_sharpe, init_guess, method='SLSQP', bounds=bounds,constraints=cons )
opt_results

     fun: -1.1481451669858183
     jac: array([ 7.21514225e-05,  5.55515289e-05,  1.22629121e-01,  1.86084658e-02,
        5.46409816e-01,  7.41941631e-02, -2.93418765e-04])
 message: 'Optimization terminated successfully.'
    nfev: 54
     nit: 6
    njev: 6
  status: 0
 success: True
       x: array([3.36384368e-01, 4.88444121e-01, 2.38524478e-18, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 1.75171512e-01])

In [89]:
if (np.sum(opt_results.x)) == 1:
    print("ok")

get_ret_vol_sr(opt_results.x)

array([0.23189315, 0.20197198, 1.14814517])

In [90]:
# create an array of possible returns  y-values
frontier_y = np.linspace(0,0.25,100)
frontier_y

array([0.        , 0.00252525, 0.00505051, 0.00757576, 0.01010101,
       0.01262626, 0.01515152, 0.01767677, 0.02020202, 0.02272727,
       0.02525253, 0.02777778, 0.03030303, 0.03282828, 0.03535354,
       0.03787879, 0.04040404, 0.04292929, 0.04545455, 0.0479798 ,
       0.05050505, 0.0530303 , 0.05555556, 0.05808081, 0.06060606,
       0.06313131, 0.06565657, 0.06818182, 0.07070707, 0.07323232,
       0.07575758, 0.07828283, 0.08080808, 0.08333333, 0.08585859,
       0.08838384, 0.09090909, 0.09343434, 0.0959596 , 0.09848485,
       0.1010101 , 0.10353535, 0.10606061, 0.10858586, 0.11111111,
       0.11363636, 0.11616162, 0.11868687, 0.12121212, 0.12373737,
       0.12626263, 0.12878788, 0.13131313, 0.13383838, 0.13636364,
       0.13888889, 0.14141414, 0.14393939, 0.14646465, 0.1489899 ,
       0.15151515, 0.1540404 , 0.15656566, 0.15909091, 0.16161616,
       0.16414141, 0.16666667, 0.16919192, 0.17171717, 0.17424242,
       0.17676768, 0.17929293, 0.18181818, 0.18434343, 0.18686

In [91]:
frontier_volatility = []
#for a possible return what is the risk
#in order to get the return/volatility combinations on the efficient frontier 
#we add the second constraint
for possible_return in frontier_y:
    cons = ({'type':'eq', 'fun':check_sum},{'type':'eq', 'fun':lambda w: (get_ret_vol_sr(w)[0]-possible_return)})
    result = minimize(minimize_volatility, init_guess, method='SLSQP', bounds=bounds, constraints=cons)
    frontier_volatility.append(result['fun'])
    print(result['fun'])
    
    
frontier_volatility
    

0.21548090574694231
0.21454982714691645
0.21361501311086978
0.21267734731659693
0.21173782520515602
0.21079756604872055
0.20985782631397967
0.20892001445676298
0.2079857072987392
0.207056668149122
0.20613486684949384
0.20197598926787166
0.1972904531255976
0.19343699109244286
0.19021439064186563
0.18717295222660063
0.1843098320795451
0.18163375294351983
0.17915299849667635
0.1768757874513699
0.17481007453137093
0.1729634377051255
0.1713429693111551
0.16995487434186315
0.16871517242266473
0.16755611138410895
0.1664758314828592
0.16547686467494968
0.1645632863690058
0.1637339481969274
0.162988231010658
0.16233298262993953
0.16176277409362838
0.1612423531574242
0.16076177382073975
0.1603210291562488
0.15992089555199362
0.15956067734056292
0.159241285513736
0.15896295177738706
0.15872587539165398
0.15853022944831172
0.15837604657855206
0.15826417375001603
0.15819179836183472
0.1581600878971263
0.1581700031680524
0.1582213498673781
0.15831369098008855
0.15844731895272113
0.15862180451734179


[0.21548090574694231,
 0.21454982714691645,
 0.21361501311086978,
 0.21267734731659693,
 0.21173782520515602,
 0.21079756604872055,
 0.20985782631397967,
 0.20892001445676298,
 0.2079857072987392,
 0.207056668149122,
 0.20613486684949384,
 0.20197598926787166,
 0.1972904531255976,
 0.19343699109244286,
 0.19021439064186563,
 0.18717295222660063,
 0.1843098320795451,
 0.18163375294351983,
 0.17915299849667635,
 0.1768757874513699,
 0.17481007453137093,
 0.1729634377051255,
 0.1713429693111551,
 0.16995487434186315,
 0.16871517242266473,
 0.16755611138410895,
 0.1664758314828592,
 0.16547686467494968,
 0.1645632863690058,
 0.1637339481969274,
 0.162988231010658,
 0.16233298262993953,
 0.16176277409362838,
 0.1612423531574242,
 0.16076177382073975,
 0.1603210291562488,
 0.15992089555199362,
 0.15956067734056292,
 0.159241285513736,
 0.15896295177738706,
 0.15872587539165398,
 0.15853022944831172,
 0.15837604657855206,
 0.15826417375001603,
 0.15819179836183472,
 0.1581600878971263,
 0.158

In [38]:
frontier_sr = frontier_y / frontier_volatility
print('\nmax sharp ratio calculated:\t\t',frontier_sr.max())

print('number of argument in array frontier_sr:\t',frontier_sr.argmax())

max_sr_ret_calc = frontier_y[frontier_sr.argmax()]
max_sr_vol_calc = frontier_volatility[frontier_sr.argmax()]
ml_sharp_ratio_annual = max_sr_ret_calc / max_sr_vol_calc

print('\n\nmax return calculated:\t\t',frontier_volatility[frontier_sr.argmax()])
print('max volatility calculated:\t',frontier_y[frontier_sr.argmax()])


max sharp ratio calculated:		 1.1481307062172934
number of argument in array frontier_sr:	 92


max return calculated:		 0.20234911501379463
max volatility calculated:	 0.23232323232323235


In [64]:
# calculated efficient frontier
plt.figure(figsize=(14,8))
plt.plot(frontier_volatility,frontier_y, 'g--');

# Turn on the minor TICKS, which are required for the minor GRID
plt.minorticks_on()
# Customize the major grid
plt.grid(which='major', linestyle='-', linewidth='0.5', color='dimgray')
# Customize the minor grid
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='gray')


#scatter-plot from monte carlo simulation
plt.scatter(vol_arr,ret_arr, c= sharpe_arr, cmap='plasma')
plt.colorbar(label='Sharpe Ratio')
plt.xlabel('Volatility')
plt.ylabel('return')

# actual portfolio Volatility/Return - red
print('\n\nactual x-value of Pf:\t\t', actual_annual_volatility)
print('actual  y-value of Pf:\t\t', actual_annual_return)
plt.scatter(actual_annual_volatility, actual_annual_return,  c='red')
plt.axhline(actual_annual_return, color='silver',linestyle='--')
plt.axvline(actual_annual_volatility, color='silver', linestyle='--');

# Monte-Carlo Volatility/Return  - blue
print('\nmonte_carlo x-value-max:\t', max_sr_vol)
print('monte_carlo y-value-max:\t', max_sr_ret)
plt.scatter(max_sr_vol, max_sr_ret,  c='blue');
plt.axhline(max_sr_ret, color='lightsteelblue',linestyle='--')
plt.axvline(max_sr_vol, color='lightsteelblue', linestyle='--');



# Machine_Learning calculated Volatility/Return - green
print('\nMachine_L x-value-max:\t\t', max_sr_vol_calc)
print('Machine_L y-value-max:\t\t', max_sr_ret_calc)
plt.scatter(max_sr_vol_calc, max_sr_ret_calc,  c='green');
plt.axhline(max_sr_ret_calc, color='gray',linestyle='--')
plt.axvline(max_sr_vol_calc, color='gray', linestyle='--');

# sharpe-Ratios
print('\n\nactual sharp-ratio-annual:\t',actual_sharp_ratio_annual)
print('monte_carlo sharp_ratio-annual:\t',mc_sharp_ratio_annual)
print('Machine_L sharp_ratio-annual:\t',ml_sharp_ratio_annual)

print('\noverall return improvement:\t',max_sr_ret_calc - actual_annual_return )
print('return improvement to actual:\t',(max_sr_ret_calc - actual_annual_return)/actual_annual_return )

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …



actual x-value of Pf:		 0.18453611700472972
actual  y-value of Pf:		 0.21652071688914692

monte_carlo x-value-max:	 0.19291445609956462
monte_carlo y-value-max:	 0.21447757136538348

Machine_L x-value-max:		 0.20234911501379463
Machine_L y-value-max:		 0.23232323232323235


actual sharp-ratio-annual:	 1.1780082837638186
monte_carlo sharp_ratio-annual:	 1.111775528396327
Machine_L sharp_ratio-annual:	 1.1481307062172934

overall return improvement:	 0.015802515434085423
return improvement to actual:	 0.07298384958782446


In [40]:
import pandas as pd

actual_weights
mc_weights = all_weights[sharp_argmax,:]
ml_weights = np.round((opt_results.x),decimals=6)

#ml_weights = opt_results.x

print(ml_weights)


w_data = {'symbol': symbol_list,
          'actual_weights': actual_weights,
          'mc_weights': mc_weights,
          'ml_weights': ml_weights,
        }
w_data

df_weights_compare = pd.DataFrame(w_data, columns = ['symbol','actual_weights','mc_weights','ml_weights'])
df_weights_compare['diff_%'] = (ml_weights - actual_weights) / actual_weights * 100

print('\n',df_weights_compare)



[0.336384 0.488444 0.       0.       0.       0.       0.175172]

   symbol  actual_weights  mc_weights  ml_weights      diff_%
0   MSFT        0.159357    0.340980    0.336384  111.088549
1    DHR        0.389121    0.399780    0.488444   25.525119
2    ATR        0.035552    0.018793    0.000000 -100.000000
3  HXGBY        0.130226    0.010682    0.000000 -100.000000
4    NVS        0.049667    0.016480    0.000000 -100.000000
5  NVZMY        0.128944    0.105742    0.000000 -100.000000
6    MCD        0.107133    0.107544    0.175172   63.508659


In [41]:
print('\nact. sharp-ratio-annual:\t',actual_sharp_ratio_annual)
print('mc sharp-ratio-annual:\t\t',mc_sharp_ratio_annual)
print('ml sharp-ratio-annual:\t\t',ml_sharp_ratio_annual)
print('difference: \t\t\t',(ml_sharp_ratio_annual-actual_sharp_ratio_annual))


act. sharp-ratio-annual:	 1.1780082837638186
mc sharp-ratio-annual:		 1.111775528396327
ml sharp-ratio-annual:		 1.1481307062172934
difference: 			 -0.029877577546525202


# ######
# CAPM
*return of stock i at time t = beta of stock i * return of market at time t + alpha of stock i at time t
## Y = m*x+b*
## return of i = beta * x + alpha*
## if beta is 1 it moves exactly as quickly as the market
## predict alpha more than 50% of the time an you can make profit#
## return of portfolio = beta of portfolio * market return + sum of (alphas of stocks *weight of i)
## beta says that there is a relation between our portfolio and the market return
## capm says alpha cannot be predicted but we will disagree with that notion

In [42]:
from scipy import stats
import pandas as pd
import numpy as np


In [43]:
# help(stats.linregress)


In [44]:
print('root_path:',root_path)
### create dataframe with portfolio adjusted close
usecols = ["timestamp", "adjusted_close"]
path_list = create_path_list(symbol_list, root_path)
pprint.pprint(path_list)
print(startd)
print(endd)

pf = retrievePF(symbol_list, path_list, startd, endd, usecols, rename_column)
print(pf.index.name)
pf.index.rename('DATE', inplace=True)
print(pf.index.name)
pf.fillna(method='ffill', inplace=True)
pf



root_path: ..
{'ATR': '../data/ATR/daily_ATR.csv',
 'DHR': '../data/DHR/daily_DHR.csv',
 'HXGBY': '../data/HXGBY/daily_HXGBY.csv',
 'MCD': '../data/MCD/daily_MCD.csv',
 'MSFT': '../data/MSFT/daily_MSFT.csv',
 'NVS': '../data/NVS/daily_NVS.csv',
 'NVZMY': '../data/NVZMY/daily_NVZMY.csv'}
2015-01-01
2020-10-09
None
DATE


,MSFT,DHR,ATR,HXGBY,NVS,NVZMY,MCD
DATE,,,,,,,
2015-01-02,41.6513,63.3710,61.2037,28.5702,75.3810,39.0758,79.6897
2015-01-05,41.2638,62.7426,59.3212,28.1195,75.5445,38.0205,78.8096
2015-01-06,40.6625,62.3139,58.6876,28.0091,74.9071,38.4268,78.9548
2015-01-07,41.1792,62.6909,59.0274,27.5032,75.2339,37.8897,80.3306
2015-01-08,42.3906,63.6594,60.7078,27.5032,77.8488,38.9171,80.6296
...,...,...,...,...,...,...,...
2020-10-02,206.1900,212.5000,112.8500,75.1500,87.5200,62.7500,222.6700
2020-10-05,210.3800,218.3500,114.5500,76.5400,88.2700,63.7600,226.0700
2020-10-06,205.9100,218.2400,114.5600,74.9250,86.4500,62.1900,224.0900


In [45]:
#from datetime import datetime, timedelta

#from myauz.myalpha_funcs import string2date, date2string

#endd = '2020-10-03'
#endd_date_object = string2date(endd)
#endd_date_object = endd_date_object + timedelta(days=1)
#endd = date2string(endd_date_object)
#pf[startd:endd]

In [46]:
invested_dict = {
    "MSFT": float(1297.59),
#   "GOOG": float(2111),
    "DHR": float(3168.48),
#   "EMR": float(418.10),
    "ATR": float(289.49),
    "HXGBY": float(1060.39),
    "NVS": float(404.42),
    "NVZMY": float(1049.95),
#   "CL": float(390.03),
    "MCD": float(872.35),  
#   "HEINY": float(575.60),     
#   "LDSVF": float(6942.25),   
}

In [47]:
total_investment, weights_dict = calc_weights(invested_dict)

print("total investment:",total_investment)

weight_sum = 0
for symbol, weight in weights_dict.items():
    pprint.pprint((symbol, weight))
    weight_sum += weight

print(weight_sum)

"""
weights = [
   .1,  #"MSFT",
   .1,  #"GOOG",    
   .1, #"DHR",
   .1,  #"EMR",
   .1,  #"ATR",
   .1,  #"HXGBY",
   .1,  #"NVS",
   .1,  #"NVZMY",    
   .1,  #"CL",
   .05,  #"MCD",    
   .03,  #"HEINY",
   .02,  #"LDSVF",  
]
"""

total investment: 8142.67
('MSFT', 0.15935682030587017)
('DHR', 0.38912052189269614)
('ATR', 0.03555222058612224)
('HXGBY', 0.1302263262541648)
('NVS', 0.049666755499117615)
('NVZMY', 0.1289441915244017)
('MCD', 0.10713316393762734)
1.0


'\nweights = [\n   .1,  #"MSFT",\n   .1,  #"GOOG",    \n   .1, #"DHR",\n   .1,  #"EMR",\n   .1,  #"ATR",\n   .1,  #"HXGBY",\n   .1,  #"NVS",\n   .1,  #"NVZMY",    \n   .1,  #"CL",\n   .05,  #"MCD",    \n   .03,  #"HEINY",\n   .02,  #"LDSVF",  \n]\n'

In [48]:
# we use log returns for algorithms especially for normalizations
#log_ret= np.log(pf/pf.shift(1))

df_returns = (pf/pf.shift(1))-1
df_cum_returns = initialize_df(symbol_list, startd, endd)
#print(df_cum_returns.head())
#print(df_returns)

df_cum_returns = (1+df_returns).cumprod()
###############achtung dataframe wird auf naechtes datum ohne NaN gesetzt
#df_cum_returns.dropna(inplace=True)
#################
df_cum_returns
#df_cum_returns.head()



,MSFT,DHR,ATR,HXGBY,NVS,NVZMY,MCD
DATE,,,,,,,
2015-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-05,0.990697,0.990084,0.969242,0.984225,1.002169,0.972994,0.988956
2015-01-06,0.976260,0.983319,0.958890,0.980361,0.993713,0.983391,0.990778
2015-01-07,0.988665,0.989268,0.964442,0.962653,0.998049,0.969646,1.008042
2015-01-08,1.017750,1.004551,0.991898,0.962653,1.032738,0.995939,1.011794
...,...,...,...,...,...,...,...
2020-10-02,4.950386,3.353269,1.843843,2.630363,1.161035,1.605853,2.794213
2020-10-05,5.050983,3.445582,1.871619,2.679015,1.170985,1.631700,2.836879
2020-10-06,4.943663,3.443847,1.871782,2.622488,1.146841,1.591522,2.812032


In [49]:
#ensure that weights vector is in correct order as symbol_list
wlist = []
for symbol in symbol_list:
    print(symbol)
    wlist.append(weights_dict[symbol])

print('wlist',wlist)
#weights = np.array(list(weights_dict.values()))
actual_weights=np.array(wlist)

print('\n',actual_weights.sum())




# pf_cum_ret = df_cum_returns * actual_weights * total_investment


pf_cum_ret = df_cum_returns * actual_weights 

pf_cum_ret.fillna(method='ffill', inplace=True, axis=1)
pf_cum_ret['TOTAL_POS'] = pf_cum_ret.sum(axis=1)
pf_cum_ret

print('\n _________________________________________________________________________________')

#pf_cum_ret = pf_cum_ret[1:]

print(pf_cum_ret['TOTAL_POS'])
pf_cum_ret.to_csv('../data/pf_cum_ret.csv')

new_endd = myauz.myalpha_funcs.date2string(pf_cum_ret.index[-1])

MSFT
DHR
ATR
HXGBY
NVS
NVZMY
MCD
wlist [0.15935682030587017, 0.38912052189269614, 0.03555222058612224, 0.1302263262541648, 0.049666755499117615, 0.1289441915244017, 0.10713316393762734]

 1.0

 _________________________________________________________________________________
DATE
2015-01-02    0.000000
2015-01-05    0.986953
2015-01-06    0.982265
2015-01-07    0.984741
2015-01-08    1.001814
                ...   
2020-10-02    3.065882
2020-10-05    3.133555
2020-10-06    3.099380
2020-10-07    3.147593
2020-10-08    3.139027
Name: TOTAL_POS, Length: 1453, dtype: float64


In [50]:

df_returns_weighted = df_returns * actual_weights
df_returns_weighted['DAILY_RETURN'] = df_returns_weighted.sum(axis=1)
df_returns_weighted = df_returns_weighted[1:]
print(df_returns_weighted.head())

return_annual_mean = df_returns_weighted['DAILY_RETURN'].mean()*252
print('\n\nAnnual Return of Portfolio:\t',return_annual_mean)

                MSFT       DHR       ATR     HXGBY       NVS     NVZMY  \
DATE                                                                     
2015-01-05 -0.001483 -0.003859 -0.001094 -0.002054  0.000108 -0.003482   
2015-01-06 -0.002322 -0.002659 -0.000380 -0.000511 -0.000419  0.001378   
2015-01-07  0.002025  0.002354  0.000206 -0.002352  0.000217 -0.001802   
2015-01-08  0.004688  0.006011  0.001012  0.000000  0.001726  0.003496   
2015-01-09 -0.001339 -0.002621 -0.000344  0.003049  0.000443 -0.000557   

                 MCD  DAILY_RETURN  
DATE                                
2015-01-05 -0.001183     -0.013047  
2015-01-06  0.000197     -0.004716  
2015-01-07  0.001867      0.002514  
2015-01-08  0.000399      0.017333  
2015-01-09 -0.001306     -0.002675  


Annual Return of Portfolio:	 0.21751599797171325


In [51]:
usecols = ["timestamp", "adjusted_close"]
spy_etf = get_daily_symbol("SPY", root_path, startd = startd, endd=endd, wd = None, usecols = usecols)
spy_etf=spy_etf[1:]
spy_etf=spy_etf.loc[startd:new_endd]
spy_etf['DAILY_RETURN'] = spy_etf['adjusted_close'].pct_change(1)

('Directory ', '../data/SPY', ' already exists')


In [52]:
#spy_etf = spy_etf[1:]
spy_etf

,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient,DAILY_RETURN
timestamp,,,,,,,,,
2015-01-05,204.17,204.3700,201.35,201.72,180.1014,169632646,0.0,1.0,NaN
2015-01-06,202.09,202.7200,198.86,199.82,178.4050,209151408,0.0,1.0,-0.009419
2015-01-07,201.42,202.7200,200.88,202.31,180.6281,125346709,0.0,1.0,0.012461
2015-01-08,204.01,206.1600,203.99,205.90,183.8334,147217784,0.0,1.0,0.017745
2015-01-09,206.40,206.4200,203.51,204.25,182.3602,158567288,0.0,1.0,-0.008014
...,...,...,...,...,...,...,...,...,...
2020-10-02,331.70,337.0126,331.19,333.84,333.8400,89431112,0.0,1.0,-0.009494
2020-10-05,336.06,339.9600,336.01,339.76,339.7600,45713108,0.0,1.0,0.017733
2020-10-06,339.91,342.1700,334.38,334.93,334.9300,90128883,0.0,1.0,-0.014216


In [53]:
spy_etf['CUM'] = spy_etf['adjusted_close'] / spy_etf['adjusted_close'].iloc[0]
spy_etf.head()

,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient,DAILY_RETURN,CUM
timestamp,,,,,,,,,,
2015-01-05,204.17,204.37,201.35,201.72,180.1014,169632646,0.0,1.0,NaN,1.000000
2015-01-06,202.09,202.72,198.86,199.82,178.4050,209151408,0.0,1.0,-0.009419,0.990581
2015-01-07,201.42,202.72,200.88,202.31,180.6281,125346709,0.0,1.0,0.012461,1.002924
2015-01-08,204.01,206.16,203.99,205.90,183.8334,147217784,0.0,1.0,0.017745,1.020722
2015-01-09,206.40,206.42,203.51,204.25,182.3602,158567288,0.0,1.0,-0.008014,1.012542


In [54]:

spy_etf.info()
df_returns_weighted.info()
pf_cum_ret.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1452 entries, 2015-01-05 to 2020-10-08
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   open               1452 non-null   float64
 1   high               1452 non-null   float64
 2   low                1452 non-null   float64
 3   close              1452 non-null   float64
 4   adjusted_close     1452 non-null   float64
 5   volume             1452 non-null   int64  
 6   dividend_amount    1452 non-null   float64
 7   split_coefficient  1452 non-null   float64
 8   DAILY_RETURN       1451 non-null   float64
 9   CUM                1452 non-null   float64
dtypes: float64(9), int64(1)
memory usage: 124.8 KB
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1452 entries, 2015-01-05 to 2020-10-08
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   MSFT          1452 non-null   fl

In [63]:
#plt.close('all')
pf_cum_ret['TOTAL_POS'].iloc[1:].plot(label='portfolio', figsize=(10,8))
spy_etf['CUM'].plot(label='SPY', figsize=(10,8))
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [56]:
#plt.close('all')
plt.scatter(df_returns_weighted['DAILY_RETURN'], spy_etf['DAILY_RETURN'], alpha=0.25)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [57]:
beta, alpha, r_value, p_value, std_err = stats.linregress(df_returns_weighted['DAILY_RETURN'].iloc[1:], spy_etf['DAILY_RETURN'].iloc[1:])

In [58]:
beta

0.9152977320944701

In [59]:
alpha

-0.0002853526008928947

In [60]:
r_value

0.8884780417816491

In [61]:
p_value

0.0

In [62]:
std_err

0.012419887375281017